<a href="https://colab.research.google.com/github/AngelSylvi/fyp_send_prototype/blob/main/Send_Prot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio transformers gtts pytesseract Pillow

In [2]:
!sudo apt-get update
!sudo apt-get install tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [3]:
import gradio as gr
from transformers import pipeline
from gtts import gTTS
import pytesseract
from PIL import Image
import io
import os
import random

In [4]:
object_classifier = pipeline("image-classification", model="google/vit-base-patch16-224")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cpu


In [5]:
# Text-to-Speech function
def text_to_speech(text, lang='en'):
    """Converts text to an audio file and returns the path."""
    tts = gTTS(text=text, lang=lang)
    temp_file = "temp_audio.mp3"
    tts.save(temp_file)
    return temp_file

In [6]:
# No need to import the pipeline for this function.
# Keep `from gtts import gTTS` and `import random`

def buddy_chat_logic(user_input):
    """
    Handles the chat input using simple keyword matching for a guaranteed response.
    """
    if not user_input or user_input.strip() == "":
        return "Hey there! What would you like to ask me?", None

    user_input_lower = user_input.lower()

    response_text = "That's an interesting question! Let's explore that together."

    # Simple keyword-based logic
    if "good morning" in user_input_lower or "hi" in user_input_lower or "hello" in user_input_lower:
        response_text = "Hi there! I'm so happy to see you. How are you today?"
    elif "sad" in user_input_lower or "unhappy" in user_input_lower or "crying" in user_input_lower or "frustrated" in user_input_lower:
        response_text = "It sounds like you're feeling a little down. It's okay. Let's take a break with a silly sound!"
        silly_things = ["Boing!", "Zzzzz!", "Wibble wobble!"]
        response_text += " " + random.choice(silly_things)
    elif "happy" in user_input_lower or "excited" in user_input_lower or "wonderful" in user_input_lower:
        response_text = "That sounds wonderful! I'm so happy for you! Let's read a story about that!"
    elif "mean" in user_input_lower:
        response_text = "I can help with that! A word's meaning is what it's trying to say. For example, 'cat' means a furry animal that purrs!"

    # Return the text and the audio file path
    try:
        audio_path = text_to_speech(response_text)
        return response_text, audio_path
    except Exception as e:
        # This catch is for the gTTS library, which can also fail.
        print(f"Error during audio generation: {e}")
        return response_text, None # Return the text without audio

# --- The rest of your code remains the same ---
# ...

In [7]:
def word_detective_logic(image_input):
    """
    Performs OCR on an image and returns the detected text.
    """
    if image_input is None:
        return "Please upload an image to detect words.", None

    try:
        # Perform OCR
        text_from_image = pytesseract.image_to_string(image_input)
    except Exception as e:
        return f"An error occurred: {e}. Please try a different image.", None

    response_text = ""
    if text_from_image.strip() == "":
        response_text = "I couldn't find any words in that picture. Let's try another one!"
    else:
        response_text = f"I found these words: {text_from_image}"

    audio_path = text_to_speech(response_text)
    return response_text, audio_path


In [8]:
def magic_camera_logic(image_input):
    """
    Identifies the main object in an image and provides a related phonics story.
    """
    if image_input is None:
        return "Please upload an image for me to see!", None

    # Use the pre-trained object classifier
    predictions = object_classifier(image_input)

    # Get the top prediction
    main_object = predictions[0]['label']

    # Simple logic for phonics story
    first_letter = main_object[0].upper()
    story = f"Aha! I see a {main_object}! '{first_letter}' is for {main_object}! Can you say '{first_letter}'? Now let's find another word that starts with that sound."

    audio_path = text_to_speech(story)
    return story, audio_path

In [9]:
def emotion_and_attention_logic(error_count, time_on_task):
    """
    This function simulates the on-device tracking for a Gradio demo.
    """
    error_count = int(error_count)
    time_on_task = int(time_on_task)

    response_text = "Everything looks good! Keep up the great work!"

    # Simple rule-based logic to simulate the "check-in"
    if error_count > 3:
        response_text = "It seems like you're getting a little stuck. Let's take a break and breathe together. Would you like to try a calming game?"
    elif time_on_task > 120: # 2 minutes
        response_text = "You've been working hard for a while now! How about we do something different for a bit?"

    audio_path = text_to_speech(response_text)
    return response_text, audio_path, 0, 0 # Reset counters after check-in

In [10]:
# --- 3. Gradio Interface (The UI part) ---

with gr.Blocks(title="Buddy's AI Reading App") as demo:
    gr.Markdown("# Buddy's AI Reading App")
    gr.Markdown("Welcome to your personal reading pal! Let's explore some features.")

    # --- 1. Buddy's Storyteller Chat ---
    with gr.Tab("Buddy's Chat"):
        chat_message = gr.Textbox(label="Talk to Buddy")
        chat_output = gr.Textbox(label="Buddy says:")
        audio_output_chat = gr.Audio(label="Buddy's Voice", autoplay=True)
        chat_button = gr.Button("Send Message")
        chat_button.click(
            fn=buddy_chat_logic,
            inputs=chat_message,
            outputs=[chat_output, audio_output_chat]
        )

    # --- 2. The Word Detective Game ---
    with gr.Tab("Word Detective"):
        with gr.Row():
            image_input_wd = gr.Image(type="pil", label="Upload a picture of a page")
            output_text_wd = gr.Textbox(label="Words I Found:")
        output_audio_wd = gr.Audio(label="Buddy Reads", autoplay=True)
        detect_button = gr.Button("Detect Words!")
        detect_button.click(
            fn=word_detective_logic,
            inputs=image_input_wd,
            outputs=[output_text_wd, output_audio_wd]
        )

    # --- 3. Buddy's Magic Camera ---
    with gr.Tab("Magic Camera"):
        with gr.Row():
            image_input_mc = gr.Image(type="pil", label="Upload a picture of an object")
            output_text_mc = gr.Textbox(label="Buddy's Story:")
        output_audio_mc = gr.Audio(label="Buddy's Voice", autoplay=True)
        camera_button = gr.Button("What is this?")
        camera_button.click(
            fn=magic_camera_logic,
            inputs=image_input_mc,
            outputs=[output_text_mc, output_audio_mc]
        )

    # --- 4. "How Are You Feeling?" Check-in ---
    with gr.Tab("Feeling Check-in"):
        gr.Markdown("This feature simulates how Buddy tracks your feelings and attention.")
        error_counter = gr.Number(value=0, label="Mistakes Made")
        time_counter = gr.Number(value=0, label="Seconds Spent on Task")
        output_text_feel = gr.Textbox(label="Buddy's Check-in:")
        output_audio_feel = gr.Audio(label="Buddy's Voice", autoplay=True)
        checkin_button = gr.Button("How am I doing, Buddy?")
        checkin_button.click(
            fn=emotion_and_attention_logic,
            inputs=[error_counter, time_counter],
            outputs=[output_text_feel, output_audio_feel, error_counter, time_counter]
        )


In [11]:
# Launch the app!
# The share=True parameter is what makes it publicly accessible.
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://675140926404130764.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
